In [1]:
import yaml

from pkg.training.optimizer import build_optimizer
from pkg.training.criterion import build_criterion
from pkg.utils.instantiate import instantiate
from pkg.training.trainer import Trainer

In [2]:
path = "./test.yaml"

# Read configuration file
with open(path, 'r') as f:
    cfg = yaml.safe_load(f)

# Print configuration
print(cfg)

{'datamodule': {'class': 'pkg.data.datamodules.ADNIDataModule', 'args': {'data': {'data_dir': '/project/aereditato/cestari/adni-mri-classification/data/preprocessing_multimodal/data/', 'scan_csv': '/project/aereditato/cestari/adni-mri-classification/data/preprocessing_multimodal/csv/dataset_multimodal.csv', 'diagnostic_csv': '/project/aereditato/cestari/adni-mri-classification/data/preprocessing_multimodal/csv/DXSUM_02Feb2026.csv', 'diagnosis': [1, 2, 3], 'tolerance': 180, 'modalities': {'MRI': ['MRI-T1-3T', 'MRI-T1-1.5T'], 'PET': ['PET-FDG']}}, 'split': {'mode': 'kfold', 'seed': 42, 'folds': 5, 'test_size': 0.1, 'val_size': 0.2}, 'loader': {'batch_size': 4, 'weighted_sampling': True}, 'transform': [{'random_flip': {'p': 0.5}}, {'random_affine': {'p': 0.5, 'degrees': 5, 'translation': 5}}, {'random_noise': {'p': 0.2, 'std': 0.02}}]}}, 'model': {'class': 'pkg.models.model_resnet18.ResNet18', 'args': {'cfg': {'init_filters': 8, 'dropout': 0.3, 'num_classes': 3}}}, 'optimizer': {'name': '

In [3]:
# Data module 
dm = instantiate(cfg["datamodule"])

# Setup 
dm.setup()

/home/cestari/.conda/envs/adni_rcc/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Matching scans to diagnoses...


100%|██████████| 2153/2153 [00:12<00:00, 166.70it/s]


Verifying scans available in dir...


13628it [00:10, 1349.22it/s]


Creating multimodal samples...


100%|██████████| 2467/2467 [00:16<00:00, 145.66it/s]


In [ ]:
# Loop over folds
for fold in range(dm.n_folds()):

    # Set current fold
    dm.set_fold(fold)

    # Build model, optimizer, and criterion
    model = instantiate(cfg["model"])

        # Create modalities from groups as described in the parameter
    optim = build_optimizer(cfg["optimizer"], model_params=model.parameters())
    criterion = build_criterion(cfg["criterion"], train_labels=dm.train_labels)

    # Build and setup task
    task = instantiate(cfg["task"])
    task.setup(model, optim, criterion)

    # Callbacks 
    callbacks = instantiate(cfg["callbacks"])

    # Build trainer 
    trainer = Trainer(task, dm, callbacks, **cfg["trainer"])

    print("Ok")
    
    # Fit model 
    trainer.fit()
    
    # Load best model 
    
    # Test

    

    


Dummy task init
Ok


TypeError: cannot unpack non-iterable NoneType object

In [8]:
df = dm.ds.df_multimodal
df

,PET,MRI,diagnosis,subject_id,strat_key,label
0,1592036.0,238627.0,1.0,002_S_0295,1.011,0
1,NaN,28560.0,1.0,002_S_0295,1.010,0
2,NaN,28561.0,1.0,002_S_0295,1.010,0
3,NaN,55275.0,1.0,002_S_0295,1.010,0
4,NaN,55276.0,1.0,002_S_0295,1.010,0
...,...,...,...,...,...,...
20805,NaN,10283169.0,1.0,941_S_7074,1.010,0
20806,NaN,11455284.0,1.0,941_S_7074,1.010,0
20807,NaN,1600180.0,2.0,941_S_7085,2.010,1
20808,NaN,1591321.0,1.0,941_S_7087,1.010,0


In [12]:
df["PET"].unique()

array([1592036.,      nan, 1592893., ..., 1603365., 1614518., 1614530.],
      shape=(2685,))

In [13]:
df["MRI"].unique()

array([ 238627.,   28560.,   28561., ..., 1600180., 1591321., 1619403.],
      shape=(20394,))

In [17]:
df[ (df["PET"].isna()) & (df["MRI"].notna())]

,PET,MRI,diagnosis,subject_id,strat_key,label
1,NaN,28560.0,1.0,002_S_0295,1.010,0
2,NaN,28561.0,1.0,002_S_0295,1.010,0
3,NaN,55275.0,1.0,002_S_0295,1.010,0
4,NaN,55276.0,1.0,002_S_0295,1.010,0
5,NaN,114209.0,1.0,002_S_0295,1.010,0
...,...,...,...,...,...,...
20805,NaN,10283169.0,1.0,941_S_7074,1.010,0
20806,NaN,11455284.0,1.0,941_S_7074,1.010,0
20807,NaN,1600180.0,2.0,941_S_7085,2.010,1
20808,NaN,1591321.0,1.0,941_S_7087,1.010,0


In [18]:
df[ (df["PET"].notna()) & (df["MRI"].notna())]

,PET,MRI,diagnosis,subject_id,strat_key,label
0,1592036.0,238627.0,1.0,002_S_0295,1.011,0
14,1592893.0,240808.0,1.0,002_S_0413,1.011,0
15,1592896.0,371991.0,1.0,002_S_0413,1.011,0
68,1592916.0,185090.0,1.0,002_S_0685,1.011,0
69,1592917.0,322437.0,1.0,002_S_0685,1.011,0
...,...,...,...,...,...,...
20739,1603499.0,996464.0,2.0,941_S_6345,2.011,1
20787,1603401.0,1310783.0,2.0,941_S_6803,2.011,1
20788,1603365.0,1291638.0,3.0,941_S_6854,3.011,2
20791,1614518.0,1467526.0,3.0,941_S_6962,3.011,2


In [19]:
df[ (df["PET"].notna()) & (df["MRI"].isna())]

,PET,MRI,diagnosis,subject_id,strat_key,label
179,1593116.0,NaN,2.0,002_S_1155,2.001,1
249,1592965.0,NaN,3.0,002_S_1268,3.001,2
322,1593121.0,NaN,3.0,002_S_4171,3.001,2
555,1593457.0,NaN,1.0,003_S_0907,1.001,0
556,1593462.0,NaN,1.0,003_S_0907,1.001,0
...,...,...,...,...,...,...
19568,1602441.0,NaN,2.0,137_S_4862,2.001,1
19918,1602572.0,NaN,2.0,141_S_6779,2.001,1
20065,1602816.0,NaN,2.0,153_S_6633,2.001,1
20069,1602868.0,NaN,3.0,153_S_6694,3.001,2
